In [1]:
from edc import print_info
print_info("geodb-sharing")


***Notebook Title***  
GeoDB: Sharing Data

***Notebook Description***  
Euro Data Cube GeoDB: Sharing Data


***Notebook Dependencies***  
This notebook requires an active subscription to:
* EDC GeoDB


In [2]:
from edc import setup_environment_variables
setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `GEODB_API_SERVER_PORT`, `GEODB_API_SERVER_URL`, `GEODB_AUTH_AUD`, `GEODB_AUTH_CLIENT_ID`, `GEODB_AUTH_CLIENT_SECRET`, `GEODB_AUTH_DOMAIN`



In [3]:
from edc import check_compatibility
check_compatibility("user-0.19.3")

This notebook is compatible with this base image version (user-0.19.3).

## Sharing Data

In [4]:
from xcube_geodb.core.geodb import GeoDBClient

In [5]:
geodb = GeoDBClient()
geodb.whoami

'geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd'

### Publish Dataset to the world

In [6]:
geodb.list_grants()

,table_name


In [7]:
geodb.publish_collection("land_use")

Access could not be granted. List grants with geodb.list_grants(){"hint":null,"details":null,"code":"42P01","message":"relation \"geodb_0c0a0ef4-8d0d-4e70-b002-033c7cbb41fd_land_use\" does not exist"}

In [8]:
geodb.list_grants()

,table_name


In [9]:
geodb.get_collection('land_use', database='public')

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,1,2020-05-11T09:05:01.715069+00:00,None,"POLYGON ((453952.629 91124.177, 453952.696 911...",4770326,1410,2019-03-26
1,2,2020-05-11T09:05:01.715069+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26
2,4,2020-05-11T09:05:01.715069+00:00,None,"POLYGON ((455929.405 97963.785, 455933.284 979...",2305596,1100,2019-02-25
3,5,2020-05-11T09:05:01.715069+00:00,None,"POLYGON ((461561.512 96119.256, 461632.114 960...",2310160,1100,2019-03-11
4,6,2020-05-11T09:05:01.715069+00:00,None,"POLYGON ((458139.092 95583.227, 458136.249 955...",3602865,1500,2019-01-05
...,...,...,...,...,...,...,...
18881,19650,2020-05-18T06:23:25.019586+00:00,None,"POLYGON ((460637.334 96865.891, 460647.927 969...",6253989,1600,2019-03-08
18882,19651,2020-05-18T06:23:25.019586+00:00,None,"POLYGON ((459467.868 96839.686, 459467.770 968...",6252044,1600,2019-03-26
18883,19652,2020-05-18T06:23:25.019586+00:00,None,"POLYGON ((459488.998 94066.248, 459498.145 940...",6245985,2000,2019-04-08
18884,19653,2020-05-18T06:23:25.019586+00:00,None,"POLYGON ((459676.680 94000.000, 459672.469 939...",6245986,2000,2019-02-20


### Revoke access

In [10]:
geodb.list_grants()

,table_name


In [11]:
geodb.unpublish_collection("land_use")

Collection already unpublished

In [12]:
geodb.list_grants()

,table_name
